#### [NE ISO](https://www.iso-ne.com/participate/support/web-services-data)

Electric Grid operator for New England (ME, NH, VT, MA, RI, CT)

1. System Load (MW)
2. Generation by Fuel Stock (5 minute)
3. Location Median Price - LMP ($/MWh)

#### [Maps](https://www.iso-ne.com/about/key-stats/maps-and-diagrams) and Blogs

1. [circuit view](https://www.iso-ne.com/static-assets/documents/100003/ems-color-system-diagram.pdf)
2. [generation and transmission](https://www.iso-ne.com/static-assets/documents/2020/04/new-england-geographic-diagram-transmission-planning.pdf)
3. [stats - use](https://www.iso-ne.com/about/key-stats/electricity-use)
4. [solar](https://www.iso-ne.com/about/what-we-do/in-depth/solar-power-in-new-england-locations-and-impact#slider1)
5. []
6. []
7. []
    

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv (
        find_dotenv (
            usecwd=True
        ),
    override=True
) # read local .env file and override any existing

from sqlalchemy import create_engine
from os import environ

username     =  environ.get("POSTGRES_USERNAME", "postgres")
password     =  environ.get("POSTGRES_PASSWORD", "postgres")
ipaddress    =  environ.get("POSTGRES_IPADDRESS", "localhost")
port         =  environ.get("POSTGRES_PORT", "5432")
dbname       =  environ.get("POSTGRES_DBNAME", "ArlingtonMA")

#establish database connection for Transform queries and Loads
cnx= create_engine(f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')

In [2]:
from requests import get
from requests.auth import HTTPBasicAuth

ne_iso_user  =  environ.get("NE_ISO_USERNAME", "YOUR_ISO_NE_USERNAME")
ne_iso_pass  =  environ.get("NE_ISO_PASSWORD", "YOUR_ISO_NE_PASSWORD")

basic = HTTPBasicAuth(ne_iso_user, ne_iso_pass)

In [3]:
import pandas as pd
from datetime import datetime, timedelta

base_url = 'https://webservices.iso-ne.com/api/v1.1/'

iso_ne_endpoints = {
    "iso_system_loads":{
        "endpoint":"fiveminutesystemload/day/{yyyymmdd}.json",
        "metric":"FiveMinSystemLoads",
        "cols":['datetime','MW','native','demand','btm_pv','native_btm_pv'],
    },
    "iso_genfuelmix":{
        "endpoint":"genfuelmix/day/{yyyymmdd}.json",
        "metric":"GenFuelMixes",
        "cols":['datetime','MW','iso_fuel_cat','iso_fuel','marginal'],
    },
    "iso_lmp":{
        "endpoint":"fiveminutelmp/day/{yyyymmdd}/location/{locationId}.json",
        "metric":"FiveMinLmps",
        "cols":['datetime','location','total','energy','congestion','loss'],
    },
}

## assign integer keys to fuel and category columns
int_value_pairs = pd.read_sql_query("select key,value from common.int_value_pairs where item='iso_fuel'",cnx)
iso_fuel = dict(zip(int_value_pairs.to_dict()['value'].values(),int_value_pairs.to_dict()['key'].values()))

int_value_pairs = pd.read_sql_query("select key,value from common.int_value_pairs where item='iso_fuel_cat'",cnx)
iso_fuel_cat = dict(zip(int_value_pairs.to_dict()['value'].values(),int_value_pairs.to_dict()['key'].values()))

## first start_date and reruns/catchups
# start_date = '2021-09-01'

## yesterday
start_date = yesterday = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
end_date   = start_date

# start_date = '2024-01-10'
# end_date   = '2024-01-11'

# data = {}
for key in iso_ne_endpoints.keys():
    df = pd.DataFrame()
    metric = iso_ne_endpoints[key]['metric']
    if key == 'iso_lmp':
        for location in list(range(4000,4009))+list(range(4010,4015))+[4017]:
            for date in pd.date_range(start_date, end_date):
                tmp = get(base_url+iso_ne_endpoints[key]['endpoint'].format(yyyymmdd=date.strftime('%Y%m%d'),locationId=location),auth=basic)
                tmp = pd.DataFrame.from_dict(tmp.json()[metric][metric[:-1]])
                tmp.columns = iso_ne_endpoints[key]['cols']
                df = pd.concat([df,tmp])

        df['location_id']=df.location.apply(lambda x: x['@LocId'])
        df = df.drop('location',axis=1)
    else:
        for date in pd.date_range(start_date, end_date):        
            tmp = get(base_url+iso_ne_endpoints[key]['endpoint'].format(yyyymmdd=date.strftime('%Y%m%d')),auth=basic)
            endie = metric[:-1]
            if key=='iso_genfuelmix':
                endie = metric[:-2]
            tmp = pd.DataFrame.from_dict(tmp.json()[metric][endie])
            tmp.columns = iso_ne_endpoints[key]['cols']
            df = pd.concat([df,tmp])
        
    df.datetime=pd.to_datetime(df.datetime).apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    
    if key =='iso_lmp':
        df = df.sort_values(['location_id','datetime']).reset_index(drop=True)
        df = df[~df.duplicated(['location_id','datetime'])]
    elif key=='iso_genfuelmix':
        df.iso_fuel=df.iso_fuel.replace(iso_fuel)
        df.iso_fuel_cat=df.iso_fuel_cat.replace(iso_fuel_cat)
        df.marginal=df.marginal.replace('N',False).replace('Y',True)
        df = df.sort_values(['iso_fuel','datetime'])
        df = df[~df.duplicated(['iso_fuel','datetime'])]
    else:
        df = df.sort_values(['datetime'])
        df = df[~df.duplicated("datetime")]


    # data[key]=df
        
    df.to_sql (
        key,
        schema='energy',
        con=cnx,
        if_exists='append',
        index=False,
)
